### Upload images for Indetification task to AWS S3

In [2]:
from PIL import Image,ImageDraw
import json
import numpy as np
from matplotlib import pyplot as plt
from math import floor
Image.MAX_IMAGE_PIXELS = None
import urllib.request, json
import requests
from io import BytesIO
import os
from urllib.parse import urlparse

import logging
import boto3
from botocore.exceptions import ClientError

# check Pillow version number
import PIL
print('Pillow Version:', PIL.__version__)
import numpy as np
import os
from pathlib import Path
import re
import json
# load and show an image with Pillow
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import matplotlib.pyplot as plt

Pillow Version: 8.1.0


### Upload each file to S3 to target bucket with folder which is the object name

In [3]:
def upload_file(file_path:os.path, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """
    #print("S3 parameters validation")
    assert file_path or bucket or object_name, "File name is empty"
    #print(file_path, bucket, object_name)


    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_path

    # Upload the file
    this_response = None
    s3 = boto3.resource('s3')


    #print("Connect to S3 API")
    try:
        with open(file_path, 'rb') as f:
            this_response = s3.Bucket('mturk-s3-cg').put_object(Key=object_name, Body=f,ContentType='image/jpeg',ACL='public-read')
        #print("Uploading to S3 is complete!", this_response)
    except ClientError as e:
        logging.error(e)
        return False
    return True

### Get configuration file details and directory path

In [14]:
path = Path(os.getcwd())
#foldername = "0001_0002"
#get new folder's storage entry folder, there on, folders likely similar ?
filedirectory = os.path.join(r'C:\Users\exx\Pictures\selected_images\300')
if os.path.exists(filedirectory):
    print("File directory to upload is here %s"%str(filedirectory))
else:
    assert os.path.exists(filedirectory),"Image files directory does not exist. Please check and give correct path. %s"%str(filedirectory)
folders = [os.path.join(filedirectory,name) for name in os.listdir(filedirectory)]
#files = []
#for d in images_directory:
#    files.extend([d.joinpath(image) for image in os.listdir(d) ])

results = dict()
this_directory = None
annotations_count = 0
file_class_count = dict()
#load json config
config_file_path =os.path.join(path.parent.parent,"config", "config.json")
if  os.path.exists(config_file_path):
    with open(config_file_path, "r") as jsonfile:
        config = json.load(jsonfile)
        print(config)
        assert config, "Config JSON has errors or is empty."

bucketname = config["bucketname"]
task2_folder = "task2/selwyn/smallbatch/"
s3_url = config["s3_url"]
print("Config file loaded. %s"%str(config_file_path))

File directory to upload is here C:\Users\exx\Pictures\selected_images\300
{'overlap': 0.2, 'tile_size': 400, 'class_thumbnail_sizes': {'bus': 171, 'van_rv': 127, 'small': 101, 'specialized': 111, 'truck': 223, 'trailer_large': 219, 'trailer_small': 101, 'unknown': 127}, 'marker_color': [200, 200, 200, 255], 'point_color': [255, 0, 0], 'bb_color': [255, 0, 0], 'bucketname': 'mturk-s3-cg', 'classify_images_folder': 'task2-images/LINZ/', 'bb_images_folder': 'task1-images/LINZ/', 's3_url': 'https://mturk-s3-cg.s3.amazonaws.com/'}
Config file loaded. C:\Users\exx\Documents\lab\code\config\config.json


In [15]:
len(folders)

2

### Upload each file to S3 

In [17]:
count = 0
for folder in folders:
    files = os.listdir(folder)
    for f in files:
        ext = os.path.splitext(f)[1]
        #we want only default content types that are defined for MTurk templates. Allowed ContentTypes by S3 are: https://s3browser.com/features-content-mime-types-editor.aspx 
        # allowed content MIME types by HTML <img> tags are https://www.iana.org/assignments/media-types/media-types.xhtml#image
        # but those allowed by MTurk HTML is supported by <img> tag but are re-drawn, painted on browser canvas 2d by MTurk crowd HTML react elements. They are also limited by size < 3mb.
        # smaller images are faster over any network so far they are <2000 pixels size i.e. 500*500 size and we want aspect ratio and resolution preserved when generating subimages. 
        # Otherwise the images take longer to load on slow internet per tests on Cross Browser Lambda Test.
        if ext in [".png", ".PNG", ".JPEG", ".jpg", ".jpeg"]:
            object_name = task2_folder+os.path.basename(folder)+"/"+os.path.basename(f)
            filename = os.path.join(folder, f)
            if upload_file(filename, bucketname, object_name):
                count += 1
    print(s3_url, folder)

https://mturk-s3-cg.s3.amazonaws.com/ C:\Users\exx\Pictures\selected_images\300\0001_0001
https://mturk-s3-cg.s3.amazonaws.com/ C:\Users\exx\Pictures\selected_images\300\0001_0002


In [18]:
count

137